In [21]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import os
import glob
import re
import configparser
import math
from collections import OrderedDict
import xlsxwriter
import shutil

In [22]:

def open_file():    

    global data

    file_path = filedialog.askopenfilename()
    file_label.config(text=file_path)
    data = pd.read_excel(file_path)

def open_file1():    

    global DMC_Code

    DMC_file_path = filedialog.askopenfilename()
    file_label_1.config(text=DMC_file_path)
    DMC_Code = pd.read_csv(DMC_file_path)
    
def split_file():

    global data
    global DMC_Code

    data[['id', 'DMC', 'temp1']] = data['chip_id'].str.split('=', expand=True)
    data[['DMC', 'Temp2']] = data['DMC'].str.split(',', expand=True)     

    DMC_Code = DMC_Code['DMC'].tolist()

    data = data.loc[data['DMC'].isin(DMC_Code)]

    data_columns = {
        #for Akari
        '3001': '20Hz',
        '3002': '35Hz',
        '3003': '80Hz',
        '3004': '300Hz',
        '3005': '900Hz',
        '3006': '1000Hz',
        '3007': '1100Hz',
        '3008': '3000Hz',
        '3009': '8000Hz',
        '3010': '10000Hz',
    }

    data_columns1 ={
        '4001': '75Hz',
        '4002': '1000Hz',
        '4003': '3000Hz',
        '4004': '10000Hz',
    }
    
    data_columns2={
        '5001': '94dBSPL',
        '5002': '100dBSPL',
        '5003': '106dBSPL',
        '5004': '112dBSPL',
        '5005': '118dBSPL',
        '5006': '124dBSPL',
        '5007': '127dBSPL',
        '5008': '130dBSPL',

    }

    with pd.ExcelWriter('AMSA.xlsx') as writer:
        for col, sheet in data_columns.items():
            data1 = data.loc[:,['site_num','DMC',f'{col};Output_{sheet}_94dBSPL_NPM;value;p']]
            data1.rename(columns={f'{col};Output_{sheet}_94dBSPL_NPM;value;p': 'SENS'}, inplace=True)

            data1.to_excel(writer, sheet_name=sheet, index=False)

            for col, sheet in data_columns1.items():
                data2 = data.loc[:,['site_num','DMC',f'{col};Phase_{sheet}_94dBSPL_NPM;value;p']]
                data2.rename(columns={f'{col};Phase_{sheet}_94dBSPL_NPM;value;p': 'PHASE'}, inplace=True)

                data2.to_excel(writer, sheet_name='Phase_'+sheet, index=False)

        for col, sheet in data_columns2.items():
            data3 = data.loc[:,['site_num','DMC',f'{col};THD_1000Hz_{sheet}_NPM;value;p']]
            data3.rename(columns={f'{col};THD_1000Hz_{sheet}_NPM;value;p': 'THD'}, inplace=True)

            data3.to_excel(writer, sheet_name='THD_'+sheet, index=False)

    tk.messagebox.showinfo("Success", "File processing completed!")

def calibration_setup_file():

    global file_path_ini

    file_path_ini = filedialog.askopenfilename()
    inifile_label.config(text=file_path_ini)
    data = pd.read_csv(file_path_ini)

def run_calib_setup_file():
    
    fpath=file_path_ini

######
    # %run A101_ini_target_generat.ipynb
    def inifile(file):
        global ini_data

        fpath=file
        
        config = configparser.ConfigParser()
        config.read(fpath)

        data = []
        for section in config.sections():
            for key in config[section]:
                value = config[section][key]
                data.append([section, key, value])

        ini_data = pd.DataFrame(data, columns=['Freq', 'Test', 'Value'])
        ini_data = ini_data.loc[(ini_data['Test'] == 'calibrationtarget') | (ini_data['Test'] == 'goldenphase'), :]

        split_data=ini_data['Freq'].str.replace('Name','Freq').str.split('=',expand=True)

        ini_data = ini_data.drop(columns='Freq',axis=1)
        split_data = split_data.drop(columns=0,axis=1)

        ini_data=pd.concat([ini_data,split_data],axis=1)
        ini_data=ini_data.rename(columns={1:'Freq'})
        
        ini_data = ini_data[ini_data['Value'] != '#'].reset_index(drop=True)
        ini_data['Value'] = ini_data['Value'].astype(float)
    
    #used to be A601_ini_target_generat.ipynb
    fpath = "Acoustic_Chambers_Calibration_Data/CalibrationSetupFile.csv"

    # Read CSV and set header using the first row
    calib_setup = pd.read_csv(fpath, header=None)
    column_list = calib_setup.iloc[0, :].tolist()
    calib_setup.columns = column_list

    # Filter and process the column list
    column_list = [round(num) for num in column_list[1:] if not math.isnan(num)]

    # based on A601 generate the list, search every frequency point, enter the each foler to find the ini file, and extract all ini file value to generate the new ini file target file.

    folder_path = os.path.dirname(fpath)

    # file name including "MicrophoneCharacterization" string
    regex = re.compile(".*MicrophoneCharacterization.*")

    writer = pd.ExcelWriter('New_ini_Target.xlsx', engine='xlsxwriter')

    # based on column list value, using glob module try to search the folder
    for value in column_list:
        search_pattern = os.path.join(folder_path, f"*{value}*")
        folders = glob.glob(search_pattern)
        if len(folders) > 0:
            for folder in folders:
                print(f"Found folder for value {value}: {folder}")
                # search the ini file from each folder
                files = glob.glob(os.path.join(folder, "*.ini"))
                if len(files) > 0:
                    for file in files:
                        # search the file naming including "MicrophoneCharacterization" string
                        match = regex.search(file)
                        if match:
                            inifile(file)
                            # write the data in  sheet 
                            sheetname=f'{value}'
                            ini_data.to_excel(writer, sheet_name=sheetname, index=False)
                else:       
                    print(f"No files found in folder {folder}")
        else:
            print(f"No folders found for value {value}")
    writer.close()
    
    

    # %run A201_AMSA1_good.ipynb
    AMSA_file_path='AMSA.xlsx'
    data=pd.read_excel(AMSA_file_path,sheet_name=None)

    writer = pd.ExcelWriter('AMSA1.xlsx', engine='xlsxwriter')

    for sheet_name,df in data.items():

        if 'SENS' in df.columns:
            df = df.groupby('site_num')['SENS'].mean().reset_index()
        elif df.columns.str.contains('PHASE').any():
            df = df.groupby('site_num')[df.columns[df.columns.str.contains('PHASE')]].mean().reset_index()
        elif df.columns.str.contains('THD').any():
            df = df.groupby('site_num')[df.columns[df.columns.str.contains('THD')]].mean().reset_index()

        sheet_name = sheet_name.replace('Hz', '')
        
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        
    writer.close()
        
    amsa_doc = pd.read_excel('AMSA1.xlsx', sheet_name=None)

    # reading ini file.
    new_ini_doc = pd.read_excel('New_ini_Target.xlsx', sheet_name=None)

    writer = pd.ExcelWriter('AMSA2.xlsx', engine='xlsxwriter')

    ordered_sheets = OrderedDict()

    # sweep all ini traget file sheet and contents
    for sheet_name, ini_sheet in new_ini_doc.items():
        print(sheet_name)
        # if sheet name is not"94", then skip
        if sheet_name != '94':
            continue

        # get all the frequency value from sheet name 
        freq_values = ini_sheet['Freq'].unique()
        # print(freq_values)

        #获取相对应的value列的读值
        for freq in freq_values:
            value = ini_sheet.loc[ini_sheet['Freq'] == freq, 'Value'].values[0]  # Freq Amplitude Target in inifile value
            value2=ini_sheet.loc[(ini_sheet['Freq'] == freq) & (ini_sheet['Test']=='goldenphase'), 'Value'].values[0]  #Phase Target in inifile value


            #  change all the freq value to be int, prepare to do searching in AMSA1
            freq=int(freq)
            Phase=f'Phase_{freq}'
            # print(freq)
            # print(type(freq))
            # print(type(Phase))

            processed = False

            for sheet_name2, amsa_sheet in amsa_doc.items():
                if str(freq)==sheet_name2 and not processed:
                    #  add new column, value is same as New_Target, but column title is frequncey value  
                    amsa_sheet.loc[:, 'New_Target'] = value

                    # each AMSA_sheet saved in indepent working sheet, using 'AMSA-{}' as working sheet name
                    sheet_name3 = '{}'.format(freq)
                    amsa_sheet.to_excel(writer, sheet_name=sheet_name3, index=False)
                    ordered_sheets[sheet_name3] = amsa_sheet
                    #marked sheet is processed 
                    processed = False

            for sheet_name2, amsa_sheet in amsa_doc.items():

                # print(sheet_name2)

                # if Phase in sheet_name2 and not processed:
                if str(Phase)==sheet_name2:
                    # print('phaseok')
                
                    amsa_sheet.loc[:, 'Phase_New_Target'] = value2

                # each AMSA_sheet saved in indepent working sheet, using 'AMSA-{}' as working sheet name
                
                    sheet_name4 = '{}'.format(Phase)
                    amsa_sheet.to_excel(writer, sheet_name=sheet_name4, index=False)
                    ordered_sheets[sheet_name4] = amsa_sheet
                    #marked sheet is processed
                    processed = True

            for sheet_name2, amsa_sheet in amsa_doc.items():
                if 'THD' in sheet_name2:
                    amsa_sheet.to_excel(writer, sheet_name=sheet_name2, index=False)
                    ordered_sheets[sheet_name2] = amsa_sheet
                else:
                    continue
    writer.close()
    
    fpath="Acoustic_Chambers_Calibration_Data/CalibrationSetupFile.csv"
    folder_path = os.path.dirname(os.path.dirname(fpath))

    calib_setup = pd.read_csv(fpath,header=None)

    column_list=list(calib_setup.loc[calib_setup[0]==1000].iloc[0,:])
    calib_setup.columns=column_list

    column_list =  [x for x in column_list if not math.isnan(x)]
    column_list=[round(num) for num in column_list]
    column_list=column_list[1::]
    
    ACC_setup_file_Path = ("Acoustic_Chambers_Calibration_Data/CalibrationSetupFile.csv")

    dir_path = os.path.dirname(ACC_setup_file_Path)

    # Generate the new ACC folder which duplitude all contents
    Old_ACC_Folder = "Acoustic_Chambers_Calibration_Data"
    New_ACC_Folder=shutil.copytree(Old_ACC_Folder, Old_ACC_Folder+'_new')
    New_ACC_Path = os.path.join(os.path.dirname(Old_ACC_Folder), 'Acoustic_Chambers_Calibration_Data_New')
    
    regex = re.compile(".*CalibSpeakersVoltageRMS.*")   # search the file in ACC folder (Speaker voltage file) 
    regex1 = re.compile(".*SystemPhase_94dBSPL.*")      # search the file in ACC folder (SystemPhase file) 

    writer1 = pd.ExcelWriter('output_spk.xlsx', engine='xlsxwriter')
    writer2 = pd.ExcelWriter('output_Phase.xlsx', engine='xlsxwriter')

    # get the each value in column_list, using glob module to find the folder which including this value 
    for value in column_list:
        search_pattern = os.path.join(New_ACC_Folder, f"*{value}*")
        folders = glob.glob(search_pattern)
        if len(folders) > 0:
            for folder in folders:
                print(f"Found folder for value {value}: {folder}")
                
                # search the file from the folderwhich get from last step.
                files = glob.glob(os.path.join(folder, "*.csv"))
                if len(files) > 0:
                    for file in files:
                        # find the file including "CalibSpeakersVoltageRMS"
                        match = regex.search(file)
                        if match and 'Real' not in file:
                            
                            print(f"Found file: {file}")

                            data=pd.read_csv(file)
                            sheet_name = str(value)

                            data.to_excel(writer1,sheet_name=sheet_name,index=False) 

                    for file in files:
                        match = regex1.search(file)
                        if match:
                            print(f"Found file: {file}")
                            data = pd.read_csv(file)
                            sheet_name = str(value)

                            data.to_excel(writer2,sheet_name=sheet_name,index=False) 
                else:
                    print(f"No files found in folder {folder}")
        else:
            print(f"No folders found for value {value}")
            
    writer1.close()
    writer2.close()
    
    #Phase compensation and ACC file system phase compensation and saved

    folder_path = "output_Phase.xlsx"
    data = pd.read_excel(folder_path, sheet_name=None)
    # print(data)

    AMSA2_Path = 'AMSA2.xlsx'
    AMSA2 = pd.read_excel(AMSA2_Path, sheet_name=None)

    if '94' in data.keys():
        data_94 = data['94'] 
        print(data_94)

        for sheet_name, sheet_data in AMSA2.items():
            if 'Phase' in sheet_name:
                match = re.search(r'\d+', sheet_name)
                if match:
                    numeric_part = match.group()
                    
                    if numeric_part in data_94.columns.values:
                        data_94_column = data_94[numeric_part]

                        # add one column in AMSA2， add Data_94_column in this new column
                        sheet_data['SystemPhase'] = data_94_column
                        sheet_data['deltaPhase']=sheet_data['PHASE']-sheet_data['Phase_New_Target']
                        sheet_data['New_SystemPhase']=sheet_data['SystemPhase']+sheet_data['deltaPhase']
                        sheet_data[numeric_part]=sheet_data['New_SystemPhase']
                        
                        #save the new spk voltage in the ACC foler - 94dBspl speaker vol
                        for root, dirs, files in os.walk(New_ACC_Path):
                            for file in files:

                                #94dBspl speaker vol updated in ACC and saved in the new ACC folder 
                                if file=='SystemPhase_94dBSPL.csv':
                                    Spk94DBSPL_file_path=os.path.join(root,file)
                                    Spk94DBSPL_data=pd.read_csv(Spk94DBSPL_file_path)

                                    if numeric_part in Spk94DBSPL_data.columns:
                                        # Spk94DBSPL_data.loc[:, sheet_name] = sheet_data['vol_ratio']*sheet_data['spkVol']
                                        Spk94DBSPL_data.loc[:, numeric_part] = sheet_data[numeric_part]
                                        Spk94DBSPL_data.to_csv(Spk94DBSPL_file_path, index=False)
                                    else:
                                        continue

    with pd.ExcelWriter(AMSA2_Path, engine='openpyxl') as writer:
        for sheet_name, sheet_data in AMSA2.items():
            sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)
            
    #amplitude compensation and ACC file Speaker voltage / RefMic MIC Amplitude compensation and saved

    folder_path_1 = "output_spk.xlsx"
    data_1 = pd.read_excel(folder_path_1, sheet_name=None)

    AMSA2_Path = 'AMSA2.xlsx'
    AMSA2 = pd.read_excel(AMSA2_Path, sheet_name=None)

    if '94' in data_1.keys():
        data_94_1 = data_1['94'] 
        print(data_94_1)


        for sheet_name, sheet_data in AMSA2.items():
            if sheet_name in data_94_1.columns.values:            
                # from data_94 get the corrlated column data.
                data_94_1_column = data_94_1[sheet_name]
            
                
                # 在AMSA2，sheet add one new column， add data_94_column to new column
                sheet_data['spkVol'] = data_94_1_column
                sheet_data['deltaSens']=sheet_data['New_Target']-sheet_data['SENS']
                sheet_data['vol_ratio']=10**(sheet_data['deltaSens']/20)
                sheet_data['New_spkVol']=sheet_data['vol_ratio']*sheet_data['spkVol']
                sheet_data[sheet_name]=sheet_data['vol_ratio']*sheet_data['spkVol']

                
                #generate the new speaker vol, save this voltage to new ACC folder - 94dBspl speaker vol file
                for root, dirs, files in os.walk(New_ACC_Path):
                    for file in files:
                        #94dBspl speaker vol updated in ACC and saved in the new ACC folder 
                        if file=='CalibSpeakersVoltageRMS_94dBSPL.csv':
                            Spk94DBSPL_file_path=os.path.join(root,file)
                            Spk94DBSPL_data=pd.read_csv(Spk94DBSPL_file_path)

                            if sheet_name in Spk94DBSPL_data.columns:
                                Spk94DBSPL_data.loc[:, sheet_name] = sheet_data[sheet_name]
                                Spk94DBSPL_data.to_csv(Spk94DBSPL_file_path, index=False)

                        #94dBspl ref mic output - amplitude value updated and saved in new ACC folder
                        if file=='ReferenceMicAmplitude_94dBSPL.csv':
                            Ref_output_file=os.path.join(root,file)
                            Ref_output_data=pd.read_csv(Ref_output_file)
                        
                            if sheet_name in Ref_output_data.columns:

                                Ref_output_data.loc[:,sheet_name]=Ref_output_data.loc[:,sheet_name]+sheet_data['deltaSens']
                                Ref_output_data.to_csv(Ref_output_file,index=False)

    #save all data in AMSA2
    with pd.ExcelWriter(AMSA2_Path, engine='openpyxl') as writer:
        for sheet_name, sheet_data in AMSA2.items():
            sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)
            
    # THD Compensation 
    AMSA2_Path = 'AMSA2.xlsx'
    AMSA2 = pd.read_excel(AMSA2_Path, sheet_name=None)


    for sheet_name, sheet_data in AMSA2.items():
        if sheet_name=='1000':
            data_94_THD = sheet_data['New_spkVol']

    for sheet_name, sheet_data in AMSA2.items():

        if 'THD' in sheet_name:
            match = re.search(r'\d+', sheet_name)
            if match:
                numeric_part = int(match.group())

                sheet_data['94dBspl'] = data_94_THD
                sheet_data['Delta_SPL'] = numeric_part - 94
                sheet_data['Sens']=10**(sheet_data['Delta_SPL']/20)*sheet_data['94dBspl']
                sheet_data[numeric_part]=sheet_data['Sens']

                for root, dirs, files in os.walk(New_ACC_Path):
                    for file in files:

                        #94dBspl speaker vol updated in ACC and saved in the new ACC folder 
                        if file=='CalibSpeakersVoltageRMS_'+str(numeric_part)+'dBSPL.csv' and file!='CalibSpeakersVoltageRMS_94dBSPL.csv' : 
                            # print(file)
                            Spk94DBSPL_file_path=os.path.join(root,file)
                            Spk94DBSPL_data=pd.read_csv(Spk94DBSPL_file_path)

                            if '1000' in Spk94DBSPL_data.columns:
                                # Spk94DBSPL_data.loc[:, sheet_name] = sheet_data['vol_ratio']*sheet_data['spkVol']
                                Spk94DBSPL_data.loc[:, '1000'] = sheet_data[numeric_part]
                                Spk94DBSPL_data.to_csv(Spk94DBSPL_file_path, index=False)
                                
    with pd.ExcelWriter(AMSA2_Path, engine='openpyxl') as writer:
        for sheet_name, sheet_data in AMSA2.items():
            sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)
    ######

    tk.messagebox.showinfo("Success", "File processing completed!")
    
def window_size():
    global window

    window = tk.Tk()
    window.wm_state('zoomed')
    window.config(background='sky blue')

    project_option()
    AMSA_Icon()
    DMC_Code_loading()
    Calib_setup_file_loading()
    
    window.mainloop()

def project_option():
    
    var=tk.StringVar(window)
    var.set('please choose project ')

    option_menu=tk.OptionMenu(window,var, 'project : Akari', 'Project : Squid', 'Project : Kassandra',)
    option_menu.config(font=("Calibri", 13))
    option_menu.place(relx=0.3,rely=0.05,relwidth=0.2, relheight=0.1)

def AMSA_Icon():
    global file_label

    #create a frame for different step
    frame = tk.Frame(bg="skyblue", bd=0, highlightthickness=1, highlightbackground="gray")
    frame.place(relx=0.04, rely=0.18, relwidth=0.8, relheight=0.2)

    # create a label
    file_label = tk.Label(text=" ", background='sky blue')
    file_label.place(relx=0.2, rely=0.2)

    # create a button 
    button_font=("Calibri", 10)
    open_button = tk.Button(text='please input GD AMSA file\n请载入GD AMSA 文件', command=open_file, font=button_font)
    open_button.place(relx=0.05, rely=0.2)

    # create a button for edit
    
    edit_button = tk.Button(text='processing AMSA file\n处理AMSA文件', command=split_file, font=button_font)
    edit_button.place(relx=0.7, rely=0.25)


def DMC_Code_loading():
    global file_label_1

    # create a label
    file_label_1 = tk.Label(text=" ", background='sky blue')
    file_label_1.place(relx=0.2, rely=0.3)

    # create a button    
    button_font=("Calibri", 10)
    open_button = tk.Button(text='please input DMC code file\n请载入DMC code 文件', command=open_file1, font=button_font)
    open_button.place(relx=0.05, rely=0.3)


def Calib_setup_file_loading():
    global inifile_label

    #create a frame for different step
    frame = tk.Frame(bg="skyblue", bd=0, highlightthickness=1, highlightbackground="gray")
    frame.place(relx=0.04, rely=0.38, relwidth=0.8, relheight=0.1)

    #creat a lable 
    inifile_label=tk.Label(text="",background='sky blue')
    inifile_label.place(relx=0.2,rely=0.4)

    #create a button for inifile loading
    button_font=("Calibri", 10) 
    open_button=  tk.Button(text='please input Calib_setup. file\n请输入 Calib_setup 文件',command=calibration_setup_file,font=button_font)
    open_button.place(relx=0.05, rely=0.4)

    # create a button for edit
    # Generate New_ini_Target file.
    edit_button = tk.Button(text='processing Calib_setup file\n处理setup文件', command=run_calib_setup_file,font=button_font)
    edit_button.place(relx=0.7, rely=0.4)


In [23]:

window_size()


Found folder for value 94: Acoustic_Chambers_Calibration_Data\Data_94dBSPL
94
Found folder for value 94: Acoustic_Chambers_Calibration_Data_new\Data_94dBSPL
Found file: Acoustic_Chambers_Calibration_Data_new\Data_94dBSPL\CalibSpeakersVoltageRMS_94dBSPL.csv
Found file: Acoustic_Chambers_Calibration_Data_new\Data_94dBSPL\SystemPhase_94dBSPL.csv
Found folder for value 100: Acoustic_Chambers_Calibration_Data_new\Data_100dBSPL
Found file: Acoustic_Chambers_Calibration_Data_new\Data_100dBSPL\CalibSpeakersVoltageRMS_100dBSPL.csv
Found folder for value 106: Acoustic_Chambers_Calibration_Data_new\Data_106dBSPL
Found file: Acoustic_Chambers_Calibration_Data_new\Data_106dBSPL\CalibSpeakersVoltageRMS_106dBSPL.csv
Found folder for value 112: Acoustic_Chambers_Calibration_Data_new\Data_112dBSPL
Found file: Acoustic_Chambers_Calibration_Data_new\Data_112dBSPL\CalibSpeakersVoltageRMS_112dBSPL.csv
Found folder for value 118: Acoustic_Chambers_Calibration_Data_new\Data_118dBSPL
Found file: Acoustic_Cham